In [ ]:
train_df = pd.read_csv('/kaggle/input/shopee-sentiment-analysis/train.csv')
test_df = pd.read_csv('/kaggle/input/shopee-sentiment-analysis/test.csv')

train_df.drop('review_id', axis=1, inplace=True)

print('Train shape:', train_df.shape)
print('Test shape:', test_df.shape)